In [ ]:
import os, sys
dir2 = os.path.abspath('')
dir1 = os.path.dirname(dir2)
# if not dir1 in sys.path: sys.path.append(dir1)

# Reset

In [ ]:
import os
try:
    os.remove("../auth/token.json")
except:
    print("No token found")

# Set Auth

In [ ]:
import os.path
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow

creds = None
SCOPES = ["https://www.googleapis.com/auth/drive.readonly"]

if os.path.exists("../auth/token.json"):
    creds = Credentials.from_authorized_user_file("../auth/token.json", SCOPES)
  # If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
  if creds and creds.expired and creds.refresh_token:
    creds.refresh(Request())
  else:
    flow = InstalledAppFlow.from_client_secrets_file(
      "../auth/credentials.json", SCOPES
    )
    creds = flow.run_local_server(port=0)
  # Save the credentials for the next run
  with open("../auth/token.json", "w") as token:
    token.write(creds.to_json())

# Demo

In [ ]:
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# If modifying these scopes, delete the file token.json.
SCOPES = ["https://www.googleapis.com/auth/drive.readonly"]


def main():
  """Shows basic usage of the Drive v3 API.
  Prints the names and ids of the first 10 files the user has access to.
  """
  creds = None
  # The file token.json stores the user's access and refresh tokens, and is
  # created automatically when the authorization flow completes for the first
  # time.
  if os.path.exists("../auth/token.json"):
    creds = Credentials.from_authorized_user_file("../auth/token.json", SCOPES)
  # If there are no (valid) credentials available, let the user log in.
  if not creds or not creds.valid:  
    if creds and creds.expired and creds.refresh_token:
      creds.refresh(Request())
    else:
      flow = InstalledAppFlow.from_client_secrets_file(
          "../auth/credentials.json", SCOPES
      )
      creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open("../auth/token.json", "w") as token:
      token.write(creds.to_json())

  try:
    service = build("drive", "v3", credentials=creds)

    # Call the Drive v3 API
    results = (
        service.files()
        .list(pageSize=10, fields="nextPageToken, files(id, name)")
        .execute()
    )

    # Pull data from Google drive api

    items = results.get("files", [])

    if not items:
      print("No files found.")
      return
    print("Files:")
    for item in items:
      print(f"{item['name']} ({item['id']})")
  except HttpError as error:
    # TODO(developer) - Handle errors from drive API.
    print(f"An error occurred: {error}")


if __name__ == "__main__":
  main()

# Search function

In [ ]:
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build

def getFileIdsInDir(query: str) -> list[list[str]]:
    # Replace 'YOUR_TOKEN.json' with the path to your token file.
    creds = Credentials.from_authorized_user_file('../auth/token.json')
    service = build('drive', 'v3', credentials=creds)

    # Specify the folder ID of the directory you want to read.

    # Define the query to retrieve files in the specified folder.
    # search for files & folder in root
    # query = f"'root' in parents"

    # Retrieve a list of files in the specified folder.
    nextPageToken = ''
    files = []
    while nextPageToken != None:
        results = service.files().list(q=query, fields="files(id, name, size), nextPageToken", spaces="drive", pageToken=nextPageToken, orderBy="name").execute()
        files = files + results.get('files', [])
        nextPageToken = results.get('nextPageToken')

    

    # Print the name and ID of each file.
    dir_ids = []
    if not files:
        print('No files found.')
        return []
    else:
        # print('Files:')
        for file in files:
            file_size = file.get('size') if file.get("size") is not None else 0
            dir_ids.append([file.get('name'), file.get('id'), int(file_size)])
    return dir_ids

In [ ]:
# Get real id of 720
# print(getFileIdsInDir(f"mimeType='application/vnd.google-apps.folder' and name='720'"))
# Search for file in 720 with gameid 7225
print(len(getFileIdsInDir(f"'1-0K7OsofHtGtXQon9kJIeqyyuYgQmhO9' in parents and name contains '17725'")))
# Search for file in hudl-game-logs with gameid 17725
print(getFileIdsInDir(f"'18tXkJNA2M_W15BfkRKTizGRb4GigZMPh' in parents and name contains '17725'"))

# Download file

In [ ]:
import io

import google.auth
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from googleapiclient.http import MediaIoBaseDownload


def download_file(real_file_id, output_dir: str):
  """Downloads a file
  Args:
      real_file_id: ID of the file to download
  Returns : IO object with location.

  Load pre-authorized user credentials from the environment.
  TODO(developer) - See https://developers.google.com/identity
  for guides on implementing OAuth2 for the application.
  """
  creds = Credentials.from_authorized_user_file('../auth/token.json')

  try:
    # create drive api client
    service = build("drive", "v3", credentials=creds)

    file_id = real_file_id

    # pylint: disable=maybe-no-member
    request = service.files().get_media(fileId=file_id)
    file = io.BytesIO()
    downloader = MediaIoBaseDownload(file, request)
    done = False
    while done is False:
      status, done = downloader.next_chunk()
      print(f"Download {int(status.progress() * 100)}.")

  except HttpError as error:
    print(f"An error occurred: {error}")
    file = None

  with open(output_dir, 'wb') as f:
    f.write(file.getvalue())



# if __name__ == "__main__":
  # download_file("1_SypSPVSFTQMklGcf1d-LW5yS2YXv15P", '../../output/first_download.md')


# Script

In [ ]:
videos_list = getFileIdsInDir(f"'1-0K7OsofHtGtXQon9kJIeqyyuYgQmhO9' in parents and name contains '17647'")
track_list = getFileIdsInDir(f"'1OCxnK8ssQTlD_osH9b3OsFI39lCIFbms' in parents and name contains '17647'")
game_log_list = getFileIdsInDir(f"'19PadFkgZA-Z5WF_lvI_fStQpfDdIMEtD' in parents and name contains '17647'")

# for video_name, video_id in videos_list: 
#     download_file(video_id, f'../data/{video_name}')

# for track_name, track_id in track_list:
#     download_file(track_id, f'../data/{track_name}')

for game_log_name, game_log_id, size in game_log_list:
    print(game_log_id)
    game_log_json = getFileIdsInDir(f"'{game_log_id}' in parents")
    print(game_log_json)
    break
    for json_name, json_id, size in game_log_json:  # implied that there should only be one file in this dir
        # download_file(json_id, f'../data/{game_log_name}.json')
        print('Download Complete')
    

In [ ]:
# videos_list = getFileIdsInDir(f"'1-0K7OsofHtGtXQon9kJIeqyyuYgQmhO9' in parents")
# track_list = getFileIdsInDir(f"'1OCxnK8ssQTlD_osH9b3OsFI39lCIFbms' in parents")
game_log_list = getFileIdsInDir(f"'19PadFkgZA-Z5WF_lvI_fStQpfDdIMEtD' in parents")



In [ ]:
game_log_list

for game_log_name, game_log_id, size in game_log_list:
    # game_log_json = getFileIdsInDir(f"'{game_log_id}' in parents")
    game_id = game_log_name.split(".")[-1]
    videos_list = getFileIdsInDir(f"'1-0K7OsofHtGtXQon9kJIeqyyuYgQmhO9' in parents and name contains '{game_id}'")
    track_list = getFileIdsInDir(f"'1OCxnK8ssQTlD_osH9b3OsFI39lCIFbms' in parents and name contains '{game_id}'")
    if len(track_list) > 4:
        # print(f"Found none matching games info for {game_id}")
        print(f"Found interesting results for {game_id}")
        print(videos_list)
        print(track_list)
        print("=============================")

In [ ]:
l = [1, 2, 3]
l.append(5)
l += l
l

In [ ]:
254.86